In [1]:
import shutil
import traceback
import json
import os
import numpy as np
import echopype as ep
from dask.distributed import Client, performance_report
from dask import delayed
from dask.utils import memory_repr
import dask

In [2]:
from pathlib import Path
import itertools as it
import datetime as dt
from dateutil import parser as dtparser
import pandas as pd
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import hvplot.xarray
import csv
import echopype as ep
import os
import warnings
from echopype import open_raw
import dask
from dask.distributed import Client
from tqdm import tqdm
import requests
from datetime import datetime
from dateutil import parser as dtparser
from importlib.metadata import version
from urllib.parse import urljoin
from datetime import timedelta


warnings.simplefilter("ignore", category=DeprecationWarning)
fs = fsspec.filesystem('https')

In [3]:
client = Client()

In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 64,Total memory: 245.77 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36837,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 245.77 GiB
Comm: tcp://127.0.0.1:46613,Total threads: 8
Dashboard: http://127.0.0.1:43853/status,Memory: 30.72 GiB
Nanny: tcp://127.0.0.1:39033,


In [5]:
fs = fsspec.filesystem('https')

In [6]:
ooi_raw_url = "https://rawdata.oceanobservatories.org/files/CE04OSPS/PC01B/ZPLSCB102_10.33.10.143/"

In [7]:
def parse_exception(exception):
    exc_dict = {
        'type': type(exception).__name__,
        'value': str(exception),
        'traceback': "".join(
            traceback.format_exception(
                type(exception), exception, exception.__traceback__
            )
        ),
    }
    return exc_dict

In [8]:
def in_range(raw_file: str, start: datetime, end: datetime) -> bool:
    ## Check if file url is in datetime range
    file_name = Path(raw_file).name
    #print('File Name:', file_name)
    
    if 'OOI-' in file_name:
        format_string = "OOI-D%Y%m%d-T%H%M%S.raw"
        file_datetime = datetime.strptime(file_name, format_string)
        #print('Parsed Datetime:', file_datetime)
        return start <= file_datetime <= end

    return False

In [9]:
start_datetime = dt.datetime(2015, 1, 1, 0, 0)
end_datetime = dt.datetime(2018, 12, 31, 23, 59)

In [10]:
desired_day_urls = []
current_date = start_datetime

while current_date <= end_datetime:
    day_url = urljoin(
        ooi_raw_url,
        f"{current_date.year}/{current_date.month:02d}/{current_date.day:02d}"
    )
    desired_day_urls.append(day_url)
    current_date += timedelta(days=1)

In [11]:
all_raw_file_urls = it.chain.from_iterable([fs.glob(f"{day_url}/*.raw") for day_url in tqdm(desired_day_urls) if  requests.get(day_url).status_code ==200 ])

100%|███████████████████████████████████████| 1461/1461 [11:35<00:00,  2.10it/s]


In [12]:
desired_raw_file_urls = list(filter(
    lambda raw_file: in_range(
        raw_file,
        start_datetime-dt.timedelta(hours=0),  # 3 hour buffer to select files
        end_datetime+dt.timedelta(hours=0)
    ),
    all_raw_file_urls
))
print(f"There are {len(desired_raw_file_urls)} raw files within the specified datetime range.")

There are 1326 raw files within the specified datetime range.


In [13]:
# base_dpath = Path('./exports')
# shutil.rmtree(base_dpath)
# base_dpath.mkdir(exist_ok=True)

# # Create a directory for all files
# output_dpath = Path(base_dpath / 'ooimooring_allfiles')
# output_dpath.mkdir(exist_ok=True)

# failed_dpath = Path('./failed')
# failed_dpath.mkdir(exist_ok=True)

In [14]:

def open_and_save(raw_file, sonar_model, use_swap, open_kwargs={}, save_kwargs={}):
    try:
        #print('here')
        ed = ep.open_raw(raw_file=raw_file, sonar_model=sonar_model, use_swap=use_swap, **open_kwargs)
#         ed.to_zarr(save_path=save_path, overwrite=overwrite, **save_kwargs)
        return ed
#         return ed.converted_raw_path
    except Exception as e:
        exc_dict = parse_exception(e)
        print("Error with Exception: ", e)
#         failed_file = failed_dpath / f"{os.path.basename(raw_file)}.txt"
#         failed_file.write_text(json.dumps(exc_dict))

In [15]:

def get_ed_list(desired_raw_file_urls):
    ed_list = []
    paths = []
    for raw_file_url in tqdm(desired_raw_file_urls):
    # Read and extract the echodata object
        ed = client.submit(open_and_save,
                       raw_file=raw_file_url,
                       sonar_model='ek60',
                       use_swap=True)
        ed_list.append(ed)
        paths.append(str(raw_file_url))
    return ed_list, paths

In [16]:
def create_year_dataframe(final_df):
    df = pd.DataFrame()
    count =0
    t = []
    for index,row in final_df.iterrows():
        #print(count)
        #final_df.at[index,'Year'] = str(row['Object_Name'].split('/')[-1].split('-')[1].split('D')[1][0:4])
        #print('here')
        df.at[count, 'Echodata_object'] = row['Echodata']
        df.at[count, 'Path'] = str(row['Object_Name'])
        df.at[count, 'Year'] = str(row['Object_Name'].split('/')[-1].split('-')[1].split('D')[1][0:4])
        df.at[count, 'Month'] = str(row['Object_Name'].split('/')[-1].split('-')[1].split('D')[1][4:6])
        channels = list(row['Echodata'].platform.channel.to_pandas())
        df.at[count, 'Channel1'] = channels[0]
        df.at[count, 'Channel2'] = channels[1]
        df.at[count, 'Channel3'] = channels[2]
        count = count +1

    print(df.shape)
    return df

In [17]:
year_df = pd.DataFrame()
year_df['Year'] = [i.split('/')[-1].split('-')[1].split('D')[1][0:4] for i in desired_raw_file_urls]
year_df['Url'] = desired_raw_file_urls
year_df.shape

(1326, 2)

In [18]:
%%time
# recursive function for combining echo data
import warnings
#warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
years = list(year_df.Year.value_counts().reset_index()['Year'])
batch_size = 50
def recur_combine_echodata(ed,batch_size):
    if len(ed)<=1:
        return ed[0]
    
    with performance_report(filename="dask-report-combine.html"):
        batches = [ed[i:i + batch_size] for i in range(0, len(ed), batch_size)]
        ed = [ep.combine_echodata(batch) for batch in tqdm(batches)]
    return recur_combine_echodata(ed,batch_size)


prefix = "echo_object"
count =0
#final_df = pd.DataFrame()
final_dict = {}
for year in years:
    print("Year :" ,year)
    year_wise_url_list = year_df.loc[year_df['Year']==year]['Url']
    ed_list, paths = get_ed_list(year_wise_url_list)

    list_of_ed = client.gather(ed_list, errors='skip')
    print('Year - ',year, len(list_of_ed))
    ed_list = list_of_ed
    ed_df = pd.DataFrame()
    ed_df['Echodata'] = ed_list
    ed_df['Object_Name'] = paths
    print(ed_df.shape)
    final_df = ed_df.dropna(axis =0)
    print(final_df.shape)
    df = create_year_dataframe(final_df)
    print('Dataframe for ', year, 'is', df.shape)
    channel_list = list(df.loc[df.Year == year]['Channel1'].value_counts().reset_index()['Channel1'])
    print(channel_list)
    for channel in channel_list:
        print('Channel',channel)
        combined_ed_list = []
        #final_df.at[count, 'Year'] = year
        ed_lists = list(df.loc[(df['Year'] == year) & (df['Channel1'] == channel)]['Echodata_object'])
        #print(ed_lists)
        print('Number of echodata objects with this channel in year ',year, 'are ', len(ed_lists))
        try:
            # Use Dask delayed to parallelize the function calls
            #combined_ed_list = dask.compute([delayed(combine_echodata)(batch) for batch in ed_lists])#[0]
            #combined_ed_list = dask.compute(delayed(combine_echodata)(ed_lists))
            combined_ed_list = recur_combine_echodata(ed_lists,batch_size)

        except Exception as e:
            print('Error with Exception:', e)
        #print(combined_ed_list)
        #inal_df.at[count, 'Echodata_object'] = combined_ed_list
        
        final_dict["".join([prefix,'_',year,'_',channel])] = combined_ed_list
        count+=1

Year : 2017


100%|███████████████████████████████████████| 728/728 [00:00<00:00, 1491.92it/s]
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/

/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value en

Year -  2017 728
(728, 2)
(714, 2)
(714, 7)
Dataframe for  2017 is (714, 7)
['GPT  38 kHz 00907208dd13 5-1 OOI.38|200', 'GPT  38 kHz 00907207b7b1 6-1 OOI.38|200']
Channel GPT  38 kHz 00907208dd13 5-1 OOI.38|200
Number of echodata objects with this channel in year  2017 are  562


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.53it/s]


...

Year : 2018


100%|█████████████████████████████████████████| 96/96 [00:00<00:00, 1475.40it/s]
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/exouser/anaconda3/lib/python3.11/

Year -  2018 96
(96, 2)
(96, 2)
(96, 7)
Dataframe for  2018 is (96, 7)
['GPT  38 kHz 00907207b7b1 6-1 OOI.38|200']
Channel GPT  38 kHz 00907207b7b1 6-1 OOI.38|200
Number of echodata objects with this channel in year  2018 are  96


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.40it/s]


CPU times: user 1min 7s, sys: 17.1 s, total: 1min 24s
Wall time: 13min 15s


In [19]:
final_dict

{'echo_object_2017_GPT  38 kHz 00907208dd13 5-1 OOI.38|200': <EchoData: standardized raw data from Internal Memory>
 Top-level: contains metadata about the SONAR-netCDF4 file format.
 ├── Environment: contains information relevant to acoustic propagation through water.
 ├── Platform: contains information about the platform on which the sonar is installed.
 │   └── NMEA: contains information specific to the NMEA protocol.
 ├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
 ├── Sonar: contains sonar system metadata and sonar beam groups.
 │   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data, including split-beam angle data when they exist.
 └── Vendor_specific: contains vendor-specific information about the sonar and the data.,
 'echo_object_2017_GPT  38 kHz 00907207b7b1 6-1 OOI.38|200': <EchoData: standardized raw data from Internal Memory>
 Top-level: contains metadata about the SONAR-netCD

In [ ]:
ed_list = []
paths = []
for raw_file_url in tqdm(desired_raw_file_urls):
    # Read and extract the echodata object
    ed = client.submit(open_and_save,
                       raw_file=raw_file_url,
                       sonar_model='ek60',
                       use_swap=True)
    ed_list.append(ed)
    paths.append(str(raw_file_url))

In [ ]:
len(paths)

In [ ]:
len(ed_list)

In [ ]:
%%time
import warnings
warnings.filterwarnings('ignore')
list_of_ed = client.gather(ed_list)
len(list_of_ed)

In [ ]:
ed_list = list_of_ed
len(ed_list)

# Old Code

ed_df = pd.DataFrame()
ed_df['Echodata'] = list_of_ed
ed_df['Object_Name'] = paths

list_of_ed = list(filter(None, list_of_ed))
len(list_of_ed)

t = list_of_ed[1].platform.channel
dict(t.to_pandas()).values()
df = pd.DataFrame()
count =0
t = []
for i in list_of_ed:
    #df.at[count, 'Path'] = str(i.converted_raw_path)
    #print(count)
    df.at[count, 'Echodata_object'] = i
    channels = list(i.platform.channel.to_pandas())
    df.at[count, 'Channel1'] = channels[0]
    df.at[count, 'Channel2'] = channels[1]
    df.at[count, 'Channel3'] = channels[2]
    count = count +1
print(df.Channel1.value_counts())
temp_df = df.loc[df['Channel1']== 'GPT  38 kHz 00907208dd13 5-1 OOI.38|200']
temp_df.head()
print(temp_df.Channel3.value_counts(dropna=False))
#filtered = list(temp_df['Path'])
new_ed_list = list(temp_df['Echodata_object'])
len(new_ed_list)

In [ ]:
ed_list[1]

# New Code

In [ ]:
ed_df = pd.DataFrame()
ed_df['Echodata'] = list_of_ed
ed_df['Object_Name'] = paths
print(ed_df.shape)
final_df = ed_df.dropna(axis =0)
print(final_df.shape)

In [ ]:
final_df['Object_Name'][0].split('/')[-1].split('-')[1].split('D')[1][0:4]

In [ ]:
%%time

#t = ed_list[1].platform.channel
#dict(t.to_pandas()).values()

#for i in ed_list:
#    print(count)
#    df.at[count, 'Echodata_object'] = i
#    #df.at[count, 'Path'] = str(i.converted_raw_path)
    #df.at[count, 'Year'] = str(i.converted_raw_path).split('/')[-1].split('-')[1].split('D')[1][0:4]
    #df.at[count, 'Month'] = str(i.converted_raw_path).split('/')[-1].split('-')[1].split('D')[1][4:6]
#    channels = list(i.platform.channel.to_pandas())
#    df.at[count, 'Channel1'] = channels[0]
#    df.at[count, 'Channel2'] = channels[1]
#    df.at[count, 'Channel3'] = channels[2]
#    count = count +1

def create_year_dataframe():
    df = pd.DataFrame()
    count =0
    t = []
    for index,row in final_df.iterrows():
        #print(count)
        #final_df.at[index,'Year'] = str(row['Object_Name'].split('/')[-1].split('-')[1].split('D')[1][0:4])
        #print('here')
        df.at[count, 'Echodata_object'] = row['Echodata']
        df.at[count, 'Path'] = str(row['Object_Name'])
        df.at[count, 'Year'] = str(row['Object_Name'].split('/')[-1].split('-')[1].split('D')[1][0:4])
        df.at[count, 'Month'] = str(row['Object_Name'].split('/')[-1].split('-')[1].split('D')[1][4:6])
        channels = list(row['Echodata'].platform.channel.to_pandas())
        df.at[count, 'Channel1'] = channels[0]
        df.at[count, 'Channel2'] = channels[1]
        df.at[count, 'Channel3'] = channels[2]
        count = count +1

    print(df.shape)
    return df

In [ ]:
%%time
# recursive function for combining echo data
years = list(df.Year.value_counts().reset_index()['Year'])
import warnings
warnings.filterwarnings('ignore')
batch_size = 50
def recur_combine_echodata(ed,batch_size):
    if len(ed)<=1:
        return ed[0]
    
    with performance_report(filename="dask-report-combine.html"):
        batches = [ed[i:i + batch_size] for i in range(0, len(ed), batch_size)]
        ed = [ep.combine_echodata(batch) for batch in tqdm(batches)]
    return recur_combine_echodata(ed,batch_size)


prefix = "echo_object"
count =0
#final_df = pd.DataFrame()
final_dict = {}
for year in years:
    print("Year :" ,year)
    channel_list = list(df.loc[df.Year == year]['Channel1'].value_counts().reset_index()['Channel1'])
    print(channel_list)
    for channel in channel_list:
        print('Channel',channel)
        combined_ed_list = []
        #final_df.at[count, 'Year'] = year
        ed_lists = list(df.loc[(df['Year'] == year) & (df['Channel1'] == channel)]['Echodata_object'])
        #print(ed_lists)
        print('Number of echodata objects with this channel in year ',year, 'are ', len(ed_lists))
        try:
            # Use Dask delayed to parallelize the function calls
            #combined_ed_list = dask.compute([delayed(combine_echodata)(batch) for batch in ed_lists])#[0]
            #combined_ed_list = dask.compute(delayed(combine_echodata)(ed_lists))
            combined_ed_list = recur_combine_echodata(ed_lists,batch_size)

        except Exception as e:
            print('Error with Exception:', e)
        #print(combined_ed_list)
        #inal_df.at[count, 'Echodata_object'] = combined_ed_list
        
        final_dict["".join([prefix,'_',year,'_',channel])] = combined_ed_list
        count+=1

In [ ]:
final_dict

In [ ]:
list(final_dict.values())

In [ ]:
ds_Sv = ep.calibrate.compute_Sv(list(final_dict.values())[0]).compute()
print(ds_Sv)

In [ ]:
ds_Sv = ep.consolidate.swap_dims_channel_frequency(ds_Sv)
ds_Sv.Sv.sel(frequency_nominal=200000).plot.pcolormesh(
    x='ping_time', cmap = 'jet', vmin=-80, vmax=-30
);

In [ ]:
client.close()